#### 第四次课

* 前半小时：讲解作业以及pandas的使用方法
* 后半小时：github与git命令讲解

In [2]:
import pandas as pd
import numpy as np
import re
import os

In [3]:
table = pd.read_csv("红楼梦.txt",encoding = "utf-8")
table.columns = ["text"]
corpus = "".join(table["text"])

In [42]:
# 切分章节
# 这个正则怎么写的？
pattern = re.compile("第[\u4e00-\u9fa5]{1,5}回\\u3000[\u4e00-\u9fa5]{1,10}\\u3000[\u4e00-\u9fa5]{1,10}")
length = len(list(re.findall(pattern,corpus)))

In [70]:
# 第一种方法:
every_chapter = re.split(pattern,corpus)[1:]

# 第二种方法：
pattern_sub = re.compile("(第[\u4e00-\u9fa5]{1,5}回\\u3000[\u4e00-\u9fa5]{1,10}\\u3000[\u4e00-\u9fa5]{1,10})(\s{4})")
corpus_anchors = re.sub('(\s)(第[\u4e00-\u9fa5]{1,5}回\\u3000[\u4e00-\u9fa5]{1,10}\\u3000[\u4e00-\u9fa5]{1,10})', 
                        r"\1我这里要替换!!!\2", corpus)
every_chapter = corpus_anchors.split("我这里要替换!!!")[1:]

In [165]:
# 找出每回最多的名字
# 在不用pandas的情况下，应该怎么做？
import jieba
from collections import Counter
import jieba.posseg

list_stopwords = []
with open("stopwords.txt","r",encoding = "utf-8") as f:
    for word in f.readlines():
        list_stopwords.append(word.strip("\n"))
        
list_corpus = []

def find_freqs(index):
    list_every_chapter = []
    for item in jieba.lcut(every_chapter[index]):
        if item not in list_stopwords:
            list_every_chapter.append(item)
    freq_dicts = dict(Counter(list_every_chapter))
    
    # 这一步用yield也可以，将这个函数变成一个generator，后方循环可以用next()语法
    return sorted(freq_dicts.items(), key=lambda d:d[1], reverse = True)

for i in range(0,len(every_chapter)): 
    list_corpus.append(find_freqs(i))

In [167]:
list_corpus[10] # 第十一回

[('凤姐儿', 63),
 ('说', 42),
 ('说道', 36),
 ('道', 24),
 ('王夫人', 20),
 ('吃', 18),
 ('罢', 18),
 ('尤氏', 16),
 ('太太', 16),
 ('秦氏', 15),
 ('听', 13),
 ('请', 12),
 ('邢夫人', 12),
 ('贾蓉', 11),
 ('老太太', 11),
 ('婶子', 11),
 ('今日', 10),
 ('园子', 10),
 ('宝玉', 10),
 ('坐', 10),
 ('太爷', 9),
 ('里', 9),
 ('母亲', 8),
 ('回', 8),
 ('尤氏道', 8),
 ('媳妇', 8),
 ('奶奶', 7),
 ('嫂子', 7),
 ('父亲', 6),
 ('贾珍', 6),
 ('老祖宗', 6),
 ('一家子', 6),
 ('爷们', 6),
 ('好些', 6),
 ('瞧瞧', 6),
 ('婆子', 6),
 ('贾瑞', 6),
 ('明日', 6),
 ('贾母', 6),
 ('日', 5),
 ('东西', 5),
 ('喜欢', 5),
 ('预备', 5),
 ('茶', 5),
 ('大爷', 5),
 ('一日', 5),
 ('老爷', 5),
 ('答应', 5),
 ('几日', 5),
 ('倒', 5),
 ('光景', 5),
 ('家里', 4),
 ('问', 4),
 ('昨日', 4),
 ('宝兄弟', 4),
 ('笑', 4),
 ('蓉', 4),
 ('两个', 4),
 ('大老爷', 4),
 ('说话', 4),
 ('哥哥', 4),
 ('回来', 4),
 ('开导', 4),
 ('走', 4),
 ('年轻', 4),
 ('这病', 4),
 ('话儿', 4),
 ('坐下', 4),
 ('点', 4),
 ('唱', 4),
 ('平儿', 4),
 ('话', 3),
 ('未敢', 3),
 ('率领', 3),
 ('前日', 3),
 ('听见', 3),
 ('侄儿', 3),
 ('日子', 3),
 ('正', 3),
 ('众', 3),
 ('热闹', 3),
 ('不肯', 3),
 ('爱', 3

In [103]:
### pandas的初步学习&学习方法

# 最基本方法
# pd.Series
# pd.DataFrame()

# 选取行、列
# df["column"]
# df.loc["columns"]

In [222]:
### 如果用pandas的情况下
DataFrame_list = []
for i in range(len(list_corpus)):
    table_every_chapter = pd.DataFrame(list_corpus[i])
    table_every_chapter.columns = ["word","freq"]
    table_every_chapter = table_every_chapter[0:100]
    DataFrame_list.append(table_every_chapter)

In [223]:
table_combined = pd.concat(DataFrame_list,axis = 1)["word"]
table_combined.columns = ["word_" + str(i) for i in range(1,121)]

In [224]:
table_combined = table_combined.T

In [225]:
# 取所有的词性，并且筛去其他词性的词
table_combined = table_combined.applymap(lambda x:jieba.posseg.lcut(x)[0])
table_combined2 = table_combined[table_combined.applymap(lambda x:"nr" in x)]

In [228]:
# 将nan全部替换成0，便于下部操作
table_combined2.fillna(0,inplace = True)

In [314]:
# 表格过滤
list_top_names = []
for i in range(1,121):
    index = "word_" + str(i)
    top_name_every_chapter = list(filter(lambda x:x != 0,table_combined2.T[index]))
    list_top_names.append(top_name_every_chapter[0:3])

In [315]:
# 最终前三个名字
table_final = pd.DataFrame(list_top_names).applymap(lambda x:str(x).strip("/nr"))

In [317]:
# 答案
table_final.head()

,0,1,2
0,士隐,那僧,那僧道
1,子兴道,老爹,小姐
2,黛玉,宝玉,王夫人
3,薛蟠,王夫人,薛家
4,宝玉,秦氏,金陵


In [318]:
# 另一种方法(写完发现其实是一样的......思路如下)
table_combined3 = table_combined2.T

# 复制range向量
copy_arraies = np.repeat([range(100,0,-1)],100,axis = 0)

# 每个词表
every_chapter_names = list(map(lambda x:table_combined3[x],table_combined3.columns))

# 做出120个表格
every_chapter_names_table = list(map(lambda x,y:pd.DataFrame([x,y]),every_chapter_names,copy_arraies))